In [40]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import soundfile as sf
import librosa as rosa
pd.set_option('display.max_colwidth', -1)

from sklearn import svm
import os
#from keras.layers import Dense,Dropout
#from keras.models import Sequential
#from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [5]:
""" Saving audio samples in List called 'sound' """

i = 0
sound = []
path = 'C:/Users/arshi/Desktop/nsynth-valid/audio/'

for filename in os.listdir(path):
    audio, rate = sf.read(path + filename)
    mfccs = rosa.feature.mfcc(audio, sr=rate)
    average = np.mean(mfccs, axis = 1)
    sound.append(average)
    print(i)
    i=i+1

In [6]:
print(len(sound))

12274


In [7]:
""" Saving classification codes - pl,wi,bo,ha - in List called 'instrument' 
 (2638) - 1 - 2638 - bass - pl
 (886) - 2639 - 2702 - brass - wi
 (470) - 2703 - 3173 - flute - wi
 (2081) - 3174 - 5255 - guitar - pl
 (2404) - 5256 - 7660 - keyboard - ha
 (663) - 7661 - 8324 - mallet - ha
 (1598) - 8325 - 9923 - organ - wi
 (720) - 9924 - 10644 - reed - wi
 (814) - 10645 - 11459 - string - bo
 
 """

instrument = []
for i in range (0, 2638):
    instrument.append('pl')
for i in range (0,886):
    instrument.append('wi')
for i in range (0,470):
    instrument.append('wi')
for i in range (0,2081):
    instrument.append('pl')
for i in range (0,2404):
    instrument.append('st')
for i in range (0,663):
    instrument.append('st')
for i in range (0,1598):
    instrument.append('wi')
for i in range (0,720):
    instrument.append('wi')
for i in range (0,814):
    instrument.append('bo')
    
print(len(instrument))


12274


In [8]:
"""Splitting between test and train"""


sound_train, sound_test, instrument_train, instrument_test = train_test_split(sound, instrument, test_size=0.3)


In [45]:
print(len(instrument_test))

3683


In [ ]:
""" Keras neural network 
https://towardsdatascience.com/k-as-in-keras-simple-classification-model-a9d2d23d5b5a

"""

model = Sequential()

#Hidden Layer 1
model.add(Dense(100,activation='relu',input_dim=8,kernel_regularization=12(0.01)))
model.add(Dropout(0.3,noise_shape=None, seed=None))

#Hidden Layer 2 
model.add(Dense(100,activation='relu',kernal_regularization=12(0.01)))
model.add(Dropout(0.3, noise_shape=None, seed=None))

#Output Layer
model.add(Dense(1,activation='sigmoid'))



In [ ]:
""" Model Compile """

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
"""Train the model"""

model_output = model.fit(sound_train, instrument_train, epochs=10, batch_size=20,verbose=1,validatondata=(sound_test,instrument_test),)

In [ ]:
print('Training accuracy :',np.mean(model_output.history["acc"]))
print('Validatoin accuracy :',np.mean(model_output.history["val_acc"]))

In [ ]:
model.save("classifier.h5")

In [43]:
""" SVM """

clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(sound_train, instrument_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [55]:
result=(clf.predict(sound_test))


In [58]:
score = 0
for i in range (0,3683):
    if result[i] == instrument_test[i]:
        score = score +1
    else:
        score = score
print(score/3683 * 100)

61.851751289709476


In [9]:
"""
 Stole Neural Network code from - https://github.com/ankonzoid/NN-from-scratch
 
 NeuralNetwork.py  (author: Anson Wong / git: ankonzoid)
"""
import math, random
import numpy as np

class NeuralNetwork:

    def __init__(self, input_dim=None, output_dim=None, hidden_layers=None, seed=1):
        if (input_dim is None) or (output_dim is None) or (hidden_layers is None):
            raise Exception("Invalid arguments given!")
        self.input_dim = input_dim # number of input nodes
        self.output_dim = output_dim # number of output nodes
        self.hidden_layers = hidden_layers # number of hidden nodes @ each layer
        self.network = self._build_network(seed=seed)

    # Train network
    def train(self, X, y, eta=0.5, n_epochs=200):
        for epoch in range(n_epochs):
            for (x_, y_) in zip(X, y):
                self._forward_pass(x_) # forward pass (update node["output"])
                yhot_ = self._one_hot_encoding(y_, self.output_dim) # one-hot target
                self._backward_pass(yhot_) # backward pass error (update node["delta"])
                self._update_weights(x_, eta) # update weights (update node["weight"])

    # Predict using argmax of logits
    def predict(self, X):
        ypred = np.array([np.argmax(self._forward_pass(x_)) for x_ in X], dtype=np.int)
        return ypred

    # ==============================
    #
    # Internal functions
    #
    # ==============================

    # Build fully-connected neural network (no bias terms)
    def _build_network(self, seed=1):
        random.seed(seed)

        # Create a single fully-connected layer
        def _layer(input_dim, output_dim):
            layer = []
            for i in range(output_dim):
                weights = [random.random() for _ in range(input_dim)] # sample N(0,1)
                node = {"weights": weights, # list of weights
                        "output": None, # scalar
                        "delta": None} # scalar
                layer.append(node)
            return layer

        # Stack layers (input -> hidden -> output)
        network = []
        if len(self.hidden_layers) == 0:
            network.append(_layer(self.input_dim, self.output_dim))
        else:
            network.append(_layer(self.input_dim, self.hidden_layers[0]))
            for i in range(1, len(self.hidden_layers)):
                network.append(_layer(self.hidden_layers[i-1], self.hidden_layers[i]))
            network.append(_layer(self.hidden_layers[-1], self.output_dim))

        return network

    # Forward-pass (updates node['output'])
    def _forward_pass(self, x):
        transfer = self._sigmoid
        x_in = x
        for layer in self.network:
            x_out = []
            for node in layer:
                node['output'] = transfer(self._dotprod(node['weights'], x_in))
                x_out.append(node['output'])
            x_in = x_out # set output as next input
        return x_in

    # Backward-pass (updates node['delta'], L2 loss is assumed)
    def _backward_pass(self, yhot):
        transfer_derivative = self._sigmoid_derivative # sig' = f(sig)
        n_layers = len(self.network)
        for i in reversed(range(n_layers)): # traverse backwards
            if i == n_layers - 1:
                # Difference between logits and one-hot target
                for j, node in enumerate(self.network[i]):
                    err = node['output'] - yhot[j]
                    node['delta'] = err * transfer_derivative(node['output'])
            else:
                # Weighted sum of deltas from upper layer
                for j, node in enumerate(self.network[i]):
                    err = sum([node_['weights'][j] * node_['delta'] for node_ in self.network[i+1]])
                    node['delta'] = err * transfer_derivative(node['output'])

    # Update weights (updates node['weight'])
    def _update_weights(self, x, eta):
        for i, layer in enumerate(self.network):
            # Grab input values
            if i == 0: inputs = x
            else: inputs = [node_['output'] for node_ in self.network[i-1]]
            # Update weights
            for node in layer:
                for j, input in enumerate(inputs):
                    # dw = - learning_rate * (error * transfer') * input
                    node['weights'][j] += - eta * node['delta'] * input

    # Dot product
    def _dotprod(self, a, b):
        return sum([a_ * b_ for (a_, b_) in zip(a, b)])

    # Sigmoid (activation function)
    def _sigmoid(self, x):
        return 1.0/(1.0+math.exp(-x))

    # Sigmoid derivative
    def _sigmoid_derivative(self, sigmoid):
        return sigmoid*(1.0-sigmoid)

    # One-hot encoding
    def _one_hot_encoding(self, idx, output_dim):
        x = np.zeros(output_dim, dtype=np.int)
        x[idx] = 1
        return x

In [29]:
"""
 NN_classifier.py  (author: Anson Wong / git: ankonzoid)
 We train a multi-layer fully-connected neural network from scratch to classify
 the seeds dataset (https://archive.ics.uci.edu/ml/datasets/seeds). An L2 loss
 function, sigmoid activation, and no bias terms are assumed. The weight
 optimization is gradient descent via the delta rule.
"""
import numpy as np
import utils as utils

def main():
    # ===================================
    # Settings
    # ===================================
    csv_filename = "data/seeds_dataset.csv"
    hidden_layers = [5] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
    eta = 0.1 # learning rate
    n_epochs = 400 # number of training epochs
    n_folds = 4 # number of folds for cross-validation
    seed_crossval = 1 # seed for cross-validation
    seed_weights = 1 # seed for NN weight initialization

    # ===================================
    # Read csv data + normalize features
    # ===================================
    #print("Reading '{}'...".format(csv_filename))
    n_classes = 4
    d = len(sound_train)
    N = len(sound_train)
    print(" -> len(sound_train) = {}, len(instrument_shape) = {}, n_classes = {}\n".format(len(sound_train), len(instrument_train), n_classes))

    print("Neural network model:")
    print(" input_dim = {}".format(d))
    print(" hidden_layers = {}".format(hidden_layers))
    print(" output_dim = {}".format(n_classes))
    print(" eta = {}".format(eta))
    print(" n_epochs = {}".format(n_epochs))
    print(" n_folds = {}".format(n_folds))
    print(" seed_crossval = {}".format(seed_crossval))
    print(" seed_weights = {}\n".format(seed_weights))

    # ===================================
    # Create cross-validation folds
    # ===================================
    idx_all = np.arange(0, N)
    idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

    # ===================================
    # Train/evaluate the model on each fold
    # ===================================
    acc_train, acc_valid = list(), list()  # training/test accuracy score
    print("Cross-validating with {} folds...".format(len(idx_folds)))
    for i, idx_valid in enumerate(idx_folds):

        # Collect training and test data from folds
        #idx_train = np.delete(idx_all, idx_valid)
        #X_train, y_train = X[idx_train], y[idx_train]
        #X_valid, y_valid = X[idx_valid], y[idx_valid]

        # Build neural network classifier model and train
        model = NeuralNetwork(input_dim=d, output_dim=n_classes,
                              hidden_layers=hidden_layers, seed=seed_weights)
        model.train(sound_train, instrument_train, eta=eta, n_epochs=n_epochs)

        # Make predictions for training and test data
        ypred_train = model.predict(X_train)
        ypred_valid = model.predict(X_valid)

        # Compute training/test accuracy score from predicted values
        acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
        acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

        # Print cross-validation result
        print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid = {})".format(
            i+1, n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

    # ===================================
    # Print results
    # ===================================
    print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(
        sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

# Driver
if __name__ == "__main__":
    main()

 -> len(sound_train) = 8591, len(instrument_shape) = 8591, n_classes = 4

Neural network model:
 input_dim = 8591
 hidden_layers = [5]
 output_dim = 4
 eta = 0.1
 n_epochs = 400
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [22]:
""" Training a model """

model = Sequential()

#Hidden Layer 1
model.add(Dense(100,activation='relu',input_dim=8,kernel_regularization=12(0.01)))
model.add(Dropout(0.3,noise_shape=None, seed=None))

#Hidden Layer 2 
model.add(Dense(100,activation='relu',kernal_regularization=12(0.01)))
model.add(Dropout(0.3, noise_shape=None, seed=None))

#Output Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [21]:
""" Predicting """

clf.predict([[sound_test]])

ValueError: Found array with dim 4. Estimator expected <= 2.

In [ ]:
"""Training a model"""

def train(data, method):
    #src_names, features, labels = unpack_data(data)
    #print 'train feature vector dim:', features.shape

    # initialize models (not all used)
    params = method[0]
    pca = decomposition.PCA(n_components=params['pca_n'])
    svc = svm.LinearSVC()
    knn = neighbors.KNeighborsClassifier(n_neighbors=params['knn_k'], metric=params['knn_metric'])

    if 'pca' in method:
        features = pca.fit_transform(features)

    if 'svc' in method:
        svc.fit(features, labels)

    if 'knn' in method:
        knn.fit(features, labels)

    return pca, svc, knn